In [1]:
import os
import fitz
from tqdm import tqdm

In [2]:
def check_rate_function(rate_1, rate_2, possible_rate):
        # 檢查兩種方法提取的評價是否相同
    if rate_1 == rate_2 and rate_1!= None:
        return rate_1
    else:
        if rate_1 in possible_rate:
            return rate_1
        elif rate_2 in possible_rate:
            return rate_2
        else :
            return 'NULL'
            

In [3]:
zip_path = r'..\file\元富'
count_true = 0
possible_rate = ['維持買進', 'BUY', '評等中立', '維持中立', 'HOLD', '中立轉買進', 
                 'Upgrade to BUY', '買進轉中立', 'Downgrade to HOLD', '評等買進', 
                 'UPGRADE TO BUY', 'Upgrade To BUY', '買進轉強力買進', '維持強力買進', 
                 'STRONG BUY', 'Upgarde to BUY']
for directory in tqdm(os.listdir(zip_path)):
    directory_path = os.path.join(zip_path, directory)
    rate_1 = None
    rate_2 = None
    rate = None
    try:
        # 使用with語句讀取PDF文件，確保在使用後自動關閉文件。
        with fitz.open(directory_path) as doc:
            mat = fitz.Matrix(1, 1)
            page = doc.load_page(0)  # 加載文檔的第0頁
            rect = page.rect
            # 檢查是否由元富投顧出版
            clip_check_source = fitz.Rect(0, rect.height/2, rect.width, rect.height)
            text_check_source = page.get_text(clip=clip_check_source)
            if '本刊載之報告為元富投顧於特定日期之分析' in text_check_source or 'MasterLink Research' in text_check_source :
                # 檢查版本
                clip_check_new_report = fitz.Rect(rect.width/2, 0, rect.width, 70)
                text_check_new_report = page.get_text(clip=clip_check_new_report)
                clip_check_old_fast_report = fitz.Rect(rect.width/2, 50, rect.width, 150)
                text_check_old_report = text_check_new_report
                text_check_old_fast_report = page.get_text(clip=clip_check_old_fast_report)
                check_old_report = ['公司訪談報告', '評等調整', '訪談報告']
                check_new_report = ['公司拜訪報告', '個股報告', '評等調升報告']
                if any(keyword in text_check_old_report for keyword in check_old_report):
                    # 報告為舊版個股報告 提取評價的第一種方法
                    try:
                        clip_old_report_1 = fitz.Rect(0, 0, rect.width, 120)
                        text_old_report_1 = page.get_text(clip=clip_old_report_1).strip()
                        pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_report_1)
                        pix.save("page-clip_old_report_1.png")
                        text_old_report_1 = text_old_report_1.split(')')[1].strip()
                        rate_1 = text_old_report_1.split('\n')[0].strip()
                    except:
                        rate_1 = None
                    # 報告為舊版個股報告 提取評價的第二種方法
                    try:
                        clip_old_report_2 = fitz.Rect(250, 60, rect.width, 110)
                        rate_2 = page.get_text(clip=clip_old_report_2).strip()
                    except:
                        rate_2 = None
                    rate = check_rate_function(rate_1, rate_2, possible_rate)
                elif '公司拜訪快報' in text_check_old_fast_report :
                # 報告為舊版公司拜訪快報 提取評價的第一種方法
                    try:
                        clip_old_fast_report_1 = fitz.Rect(0, 0, rect.width, 200)
                        text_old_fast_report_1 = page.get_text(clip=clip_old_fast_report_1).strip()
                        text_old_fast_report_1 = text_old_fast_report_1.split('建議：')[1].strip()
                        rate_1 = text_old_fast_report_1.split('\n')[0].strip()
                    except:
                        rate_1 = None
                    # 報告為舊版公司拜訪快報 提取評價的第二種方法
                    try:
                        clip_old_fast_report_2 = fitz.Rect(70, 140, rect.width, 170)
                        rate_2 = page.get_text(clip=clip_old_fast_report_2).strip()
                    except:
                        rate_2 = None
                    rate = check_rate_function(rate_1, rate_2, possible_rate)
                elif any(keyword in text_check_new_report for keyword in check_new_report):
                    # 報告為新版個股報告 提取評價的第一種方法
                    try:
                        clip_new_report_1 = fitz.Rect(0, 0, rect.width, 120)
                        text_new_report_1 = page.get_text(clip=clip_new_report_1).strip()
                        text_new_report_1 = text_new_report_1.split(')')[1].strip()
                        rate_1 = text_new_report_1.split('\n')[0].strip()
                    except:
                        rate_1 = None
                    # 報告為新版個股報告 提取評價的第二種方法
                    try:
                        clip_new_report_2 = fitz.Rect(250, 60, rect.width, 110)
                        rate_2 = page.get_text(clip=clip_new_report_2).strip()
                    except:
                        rate_2 = None
                    rate = check_rate_function(rate_1, rate_2, possible_rate)
                elif '公司拜訪快報' in text_check_new_report :
                    # 報告為新版公司拜訪快報 提取評價的第一種方法
                    try:
                        clip_new_fast_report_1 = fitz.Rect(0, 0, rect.width, 150)
                        text_new_fast_report_1 = page.get_text(clip=clip_new_fast_report_1).strip()
                        text_new_fast_report_1 = text_new_fast_report_1.split('建議：')[1].strip()
                        rate_1 = text_new_fast_report_1.split('\n')[0].strip()
                    except:
                        rate_1 = None
                    # 報告為新版公司拜訪快報 提取評價的第二種方法
                    try:
                        clip_new_fast_report_2 = fitz.Rect(80, 105, rect.width, 140)
                        rate_2 = page.get_text(clip=clip_new_fast_report_2).strip()
                    except:
                        rate_2 = None
                    rate = check_rate_function(rate_1, rate_2, possible_rate)

                elif 'Company Report' in text_check_new_report :
                    # 報告為英文版個股報告 提取評價的第一種方法
                    try:
                        clip_english_report_1 = fitz.Rect(0, 0, rect.width, 100)
                        text_english_report_1 = page.get_text(clip=clip_english_report_1).strip()
                        text_english_report_1 = text_english_report_1.split(')')[-1].strip()
                        rate_1 = text_english_report_1.split('\n')[0].strip()
                    except:
                        rate_1 = None
                    # 報告為新版個股報告 提取評價的第二種方法
                    try:
                        clip_english_report_2 = fitz.Rect(320, 60, rect.width, 110)
                        rate_2 = page.get_text(clip=clip_english_report_2).strip()
                    except:
                        rate_2 = None
                    rate = check_rate_function(rate_1, rate_2, possible_rate)
                
                elif '晨訊' in  text_check_new_report :
                    rate = '晨訊'
                else:
                    rate = 'wrong'
                    # print(f'{directory_path} is not 個股報告 or 拜訪快報')
            else:
                # False
                print(f'{directory_path} is not posted by 元富投顧')
    except:
        print(directory_path, 'fail')
    if rate == 'NULL' :
        print(directory_path, rate,
                f'rate_1={rate_1}, rate_2={rate_2}')
    elif rate == 'wrong':
        # pass
        print(directory_path, rate,
              f'text_check_new_report={text_check_new_report}, text_check_old_fast_report={text_check_old_fast_report}')
    elif rate != None :
        count_true += 1
        # print(directory_path, rate,
        #         f'rate_1={rate_1}, rate_2={rate_2}')
    else :
        print(directory_path, rate,
                f'rate_1={rate_1}, rate_2={rate_2}')
# 計算成功和失敗的數量並打印
count_false = len(os.listdir(zip_path)) - count_true
success_rate = round((count_true/len(os.listdir(zip_path))), 3)*100
print(f'共{len(os.listdir(zip_path))}件, 成功{count_true}件, 失敗{count_false}件, 成功率:{success_rate}%')

  8%|▊         | 169/2216 [00:21<04:15,  8.00it/s]

..\file\元富\1710_東聯_2018-09-04_元富_1710東聯20180904-報告.pdf wrong text_check_new_report=, text_check_old_fast_report=


  9%|▉         | 210/2216 [00:26<02:59, 11.20it/s]

..\file\元富\1789_神隆_2018-02-26_元富_1789神隆20180226-報告.pdf wrong text_check_new_report=, text_check_old_fast_report=


 10%|▉         | 218/2216 [00:27<03:09, 10.55it/s]

...

..\file\元富\8464_億豐_2018-08-17_元富_8464億豐20180817-報告.pdf wrong text_check_new_report=, text_check_old_fast_report=
..\file\元富\8464_億豐_2018-08-20_元富_8464億豐20180817-報告.pdf wrong text_check_new_report=, text_check_old_fast_report=


 99%|█████████▉| 2194/2216 [03:49<00:01, 11.57it/s]

..\file\元富\9939_宏全_2018-08-14_元富_9939宏全20180814-報告.pdf wrong text_check_new_report=, text_check_old_fast_report=
..\file\元富\9939_宏全_2018-08-15_元富_9939宏全20180814-報告.pdf wrong text_check_new_report=, text_check_old_fast_report=


100%|██████████| 2216/2216 [03:51<00:00,  9.57it/s]

共2216件, 成功2176件, 失敗40件, 成功率:98.2%


In [17]:
import fitz
doc = fitz.open(r'..\file\元富\6598_ABC_元富_HOLD.pdf')
mat = fitz.Matrix(1, 1)
page = doc.load_page(0)  # loads page number 'pno' of the document (0-based)
rect = page.rect
pix = page.get_pixmap(matrix=mat, alpha=False)
pix.save("page-%i.png" % page.number)

In [18]:
# 檢查是否由元富投顧出版
clip_check_source = fitz.Rect(0, rect.height/2, rect.width, rect.height)
pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_check_source)
pix.save("page-clip_check_source.png")
text_check_source = page.get_text(clip=clip_check_source)
if '本刊載之報告為元富投顧於特定日期之分析' in text_check_source or 'MasterLink Research'in text_check_source:
    print('True')

True


In [19]:
# 檢查版本
clip_check_new_report = fitz.Rect(rect.width/2, 0, rect.width, 70)
pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_check_new_report)
pix.save("page-clip_check_new_report.png")
text_check_new_report = page.get_text(clip=clip_check_new_report)

clip_check_old_fast_report = fitz.Rect(rect.width/2, 50, rect.width, 150)
pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_check_old_fast_report)
pix.save("page-clip_check_old_report.png")
text_check_old_report = text_check_new_report
text_check_old_fast_report = page.get_text(clip=clip_check_old_fast_report)

if '公司訪談報告' in text_check_old_report or '評等調整' in text_check_old_report or '訪談報告' in text_check_old_report :
    print('舊版個股報告')
elif'公司拜訪快報' in text_check_old_fast_report :
    print('舊版公司拜訪快報')
elif '公司拜訪報告' in text_check_new_report or '個股報告' in text_check_new_report or '評等調升報告' in text_check_new_report or 'Company Report' in text_check_new_report:
    print('新版個股報告')
elif '公司拜訪快報' in text_check_new_report :
    print('新版公司拜訪快報')
else :
    print('NULL')

新版個股報告


In [20]:
text_check_old_report

'Company Report \n'

In [21]:
# 報告為舊版個股報告 提取評價的第一種方法
try:
    clip_old_report_1 = fitz.Rect(0, 0, rect.width, 100)
    text_old_report_1 = page.get_text(clip=clip_old_report_1).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_report_1)
    pix.save("page-clip_old_report_1.png")
    text_old_report_1 = text_old_report_1.split(')')[-1].strip()
    rate_1 = text_old_report_1.split('\n')[0].strip()
except:
    rate_1 = None
rate_1

'HOLD'

In [22]:
# 報告為舊版個股報告 提取評價的第二種方法
try:
    clip_old_report_2 = fitz.Rect(310, 60, rect.width, 110)
    rate_2 = page.get_text(clip=clip_old_report_2).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_report_2)
    pix.save("page-clip_old_report_2.png")
except:
    rate_2 = None
rate_2

'8 TT) \nHOLD'

In [23]:
# 報告為舊版公司拜訪快報 提取評價的第一種方法
try:
    clip_old_fast_report_1 = fitz.Rect(0, 0, rect.width, 200)
    text_old_fast_report_1 = page.get_text(clip=clip_old_fast_report_1).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_fast_report_1)
    pix.save("page-clip_old_fast_report_1.png")
    text_old_fast_report_1 = text_old_fast_report_1.split('建議：')[1].strip()
    rate_1 = text_old_fast_report_1.split('\n')[0].strip()
except:
    rate_1 = None
rate_1

In [24]:
# 報告為舊版公司拜訪快報 提取評價的第二種方法
try:
    clip_old_fast_report_2 = fitz.Rect(70, 140, rect.width, 170)
    rate_2 = page.get_text(clip=clip_old_fast_report_2).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_fast_report_2)
    pix.save("page-clip_old_fast_report_2.png")
except:
    rate_2 = None
rate_2

'lyst: Irene Huang \nfungus product is expected to start making sales contributions in \n2H23. Among the products developed by Applied BioCode (ABC-KY),'

In [25]:
# 報告為新版個股報告 提取評價的第一種方法
try:
    clip_new_report_1 = fitz.Rect(0, 0, rect.width, 120)
    text_new_report_1 = page.get_text(clip=clip_new_report_1).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_new_report_1)
    pix.save("page-clip_new_report_1.png")
    text_new_report_1 = text_new_report_1.split(')')[1].strip()
    rate_1 = text_new_report_1.split('\n')[0].strip()
except:
    rate_1 = None
rate_1

'HOLD'

In [26]:
# 報告為新版個股報告 提取評價的第二種方法
try:
    clip_new_report_2 = fitz.Rect(250, 60, rect.width, 110)
    rate_2 = page.get_text(clip=clip_new_report_2).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_new_report_2)
    pix.save("page-clip_new_report_2.png")
except:
    rate_2 = None
rate_2

'p. (6598 TT) \nHOLD'

In [27]:
rate_1 == rate_2

False

In [28]:
# 報告為新版公司拜訪快報 提取評價的第一種方法
try:
    clip_new_fast_report_1 = fitz.Rect(0, 0, rect.width, 150)
    text_new_fast_report_1 = page.get_text(clip=clip_new_fast_report_1).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_new_fast_report_1)
    pix.save("page-clip_new_fast_report_1.png")
    text_new_fast_report_1 = text_new_fast_report_1.split('建議：')[1].strip()
    rate_1 = text_new_fast_report_1.split('\n')[0].strip()
except:
    rate_1 = None
rate_1

In [29]:
# 報告為新版公司拜訪快報 提取評價的第二種方法
try:
    clip_new_fast_report_2 = fitz.Rect(80, 105, rect.width, 140)
    rate_2 = page.get_text(clip=clip_new_fast_report_2).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_new_fast_report_2)
    pix.save("page-clip_new_fast_report_2.png")
except:
    rate_2 = None
rate_2

'□ TW Mid-Cap100  □ MSCI'

In [30]:
if 'Company Report' in text_check_new_report :
    print('英文版個股報告')

英文版個股報告


In [31]:
# 報告為英文版個股報告 提取評價的第一種方法
try:
    clip_english_report_1 = fitz.Rect(0, 0, rect.width, 100)
    text_english_report_1 = page.get_text(clip=clip_english_report_1).strip()
    text_english_report_1 = text_english_report_1.split(')')[-1].strip()
    rate_1 = text_english_report_1.split('\n')[0].strip()
except:
    rate_1 = None
rate_1

'HOLD'

In [36]:
# 報告為英文版個股報告 提取評價的第二種方法
try:
    clip_english_report_2 = fitz.Rect(370, 60, rect.width, 110)
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_english_report_2)
    pix.save("page-clip_english_report_2.png")
    rate_2 = page.get_text(clip=clip_english_report_2).strip()
except:
    rate_2 = None
rate_2

'HOLD'